In [1]:
import requests
from bs4 import BeautifulSoup
import time
import openpyxl

In [3]:
# The url of IMDb top rated movies
url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
main_page = requests.get(url)
main_page.status_code

200

In [4]:
# html parser method
soup = BeautifulSoup(main_page.content, 'html.parser')
links = soup.find('ul', class_='quicklinks').find_all('li')

genre_urls = []

# get all genres urls
for i in range(len(links)):
    genre = links[i].find('a').get_text().split()[0]
    url = 'https://imdb.com' + links[i].find('a',href = True)['href']
    genre_urls.append((genre, url))

genre_urls[0]

('Action',
 'https://imdb.com/search/title?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,')

In [5]:
# Example of action genre
action_url = genre_urls[0][1]
action = requests.get(action_url)
action.status_code


200

In [6]:
# Example of action genre 1st page 
# Scrap the information about the first movie

info = []
index = 4

action_soup = BeautifulSoup(action.content, 'html.parser')
action_moives = action_soup.find('div', class_='lister-list').find_all(class_='lister-item mode-advanced')
rank = int(''.join(action_moives[index].find(class_='lister-item-content').find('span', class_='lister-item-index unbold text-primary').get_text()[:-1].split(',')))
title = action_moives[index].find(class_='lister-item-content').find('a').get_text()
year = int(action_moives[index].find(class_='lister-item-content').find('span', class_='lister-item-year text-muted unbold').get_text()[-5:-1])
Certificate = action_moives[index].find(class_='lister-item-content').find('p', class_='text-muted').find('span',class_='certificate')
if not Certificate:
    certificate = ' '
else:
    certificate = Certificate.get_text()
runtime = action_moives[index].find(class_='lister-item-content').find('span', class_='runtime').get_text()
genre = action_moives[index].find(class_='lister-item-content').find('span', class_='genre').get_text().rstrip()[1:]
rating = float(action_moives[index].find(class_='lister-item-content').find('strong').get_text())
Metascore = action_moives[index].find(class_='lister-item-content').find('span', class_='metascore favorable')
if not Metascore:
    metascore = ' '
else:
    metascore = int(Metascore.get_text().rstrip())
intro = action_moives[index].find(class_='lister-item-content').find_all('p', class_='text-muted')[1].get_text()[1:]
votes = int(action_moives[index].find(class_='lister-item-content').find('p', class_= 'sort-num_votes-visible').find_all('span')[1]['data-value'])
Gross = len(action_moives[index].find(class_='lister-item-content').find('p', class_= 'sort-num_votes-visible').find_all('span'))
if Gross == 5:
    gross = int(''.join(action_moives[index].find(class_='lister-item-content').find('p', class_= 'sort-num_votes-visible').find_all('span')[-1]['data-value'].split(',')))
else:
    gross = ' '
URL = 'https://imdb.com'+action_moives[index].find(class_='lister-item-content').find('h3', class_='lister-item-header').find('a', href = True, text=True)['href']

info = [rank, title, year, certificate, runtime, genre, rating, metascore, votes, gross, URL, intro]

rank



5

In [7]:
excel = openpyxl.Workbook()
sheet = excel.active
sheet.title = 'IMDb Movies Database'
sheet.append(['Rank', 'Title', 'Year', 'Certificate', 'Runtime', 'Genre', 'Rating', 'Metascore', 'Votes', 'Gross', 'URL', 'Intro'])
sheet.append([rank, title, year, certificate, runtime, genre, rating, metascore, votes, gross, URL, intro])

excel.save('IMDb_example1.xlsx')

In [8]:
# Scrap the Action genre movies
# Get the next page url

last2_url ='https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start=1500&ref_=adv_prv'
last2 = requests.get(last2_url)
last2_soup = BeautifulSoup(last2.content, 'html.parser')
page = last2_soup.find('div', class_='redesign').find_all(class_='desc')[0].find('a', class_='lister-page-next next-page') or 0
if not page:
    next_page = ''
else:
    next_page = 'https://imdb.com'+last2_soup.find('div', class_='redesign').find_all(class_='desc')[0].find('a', class_='lister-page-next next-page')['href']
next_page

'https://imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start=1550'

In [9]:
def get_info(url):
    excel = openpyxl.Workbook()
    sheet = excel.active
    sheet.title = 'IMDb Movies Database'
    sheet.append(['Rank', 'Title', 'Year', 'Certificate', 'Runtime', 'Genre', 'Rating', 'Metascore', 'Votes', 'Gross', 'URL', 'Intro'])
    
    url_request = requests.get(url)
    soup = BeautifulSoup(url_request.content, 'html.parser')
    movies = soup.find('div', class_='lister-list').find_all(class_='lister-item mode-advanced')

    for index in range(len(movies)):
        rank = int(action_moives[index].find(class_='lister-item-content').find('span', class_='lister-item-index unbold text-primary').get_text()[:-1])
        title = action_moives[index].find(class_='lister-item-content').find('a').get_text()
        year = int(action_moives[index].find(class_='lister-item-content').find('span', class_='lister-item-year text-muted unbold').get_text()[-5:-1])
        print(title)
        Certificate = action_moives[index].find(class_='lister-item-content').find('p', class_='text-muted').find('span',class_='certificate')
        if not Certificate:
            certificate = ' '
        else:
            certificate = Certificate.get_text()
        runtime = action_moives[index].find(class_='lister-item-content').find('span', class_='runtime').get_text()
        genre = action_moives[index].find(class_='lister-item-content').find('span', class_='genre').get_text().rstrip()[1:]
        rating = float(action_moives[index].find(class_='lister-item-content').find('strong').get_text())
        Metascore = action_moives[index].find(class_='lister-item-content').find('span', class_='metascore favorable')
        if not Metascore:
            metascore = ' '
        else:
            metascore = int(Metascore.get_text().rstrip())
        intro = action_moives[index].find(class_='lister-item-content').find_all('p', class_='text-muted')[1].get_text()[1:]
        votes = int(action_moives[index].find(class_='lister-item-content').find('p', class_= 'sort-num_votes-visible').find_all('span')[1]['data-value'])
        Gross = len(action_moives[index].find(class_='lister-item-content').find('p', class_= 'sort-num_votes-visible').find_all('span'))
        if Gross == 5:
            gross = int(''.join(action_moives[index].find(class_='lister-item-content').find('p', class_= 'sort-num_votes-visible').find_all('span')[-1]['data-value'].split(',')))
        else:
            gross = ' '
        URL = 'https://imdb.com'+action_moives[index].find(class_='lister-item-content').find('h3', class_='lister-item-header').find('a', href = True, text=True)['href']
        sheet.append([rank, title, year, certificate, runtime, genre, rating, metascore, votes, gross, URL, intro])
    excel.save('IMDB_example2.xlsx')



In [10]:
url_request = requests.get(genre_urls[0][1])
soup = BeautifulSoup(url_request.content, 'html.parser')
movies = soup.find('div', class_='lister-list').find_all(class_='lister-item mode-advanced')
len(movies)


50

In [14]:
def scrap(url):
    excel = openpyxl.load_workbook('IMDb_database.xlsx')
    sheet = excel.active

 
    while url != 0:
        url_request = requests.get(url)
        url_soup = BeautifulSoup(url_request.content, 'html.parser')
        movies = url_soup.find('div', class_='lister-list').find_all(class_='lister-item mode-advanced')
        print(url)
        for index in range(len(movies)):
            rank = int(''.join(movies[index].find(class_='lister-item-content').find('span', class_='lister-item-index unbold text-primary').get_text()[:-1].split(',')))
            title = movies[index].find(class_='lister-item-content').find('a').get_text()
            year = int(movies[index].find(class_='lister-item-content').find('span', class_='lister-item-year text-muted unbold').get_text()[-5:-1])
            Certificate = movies[index].find(class_='lister-item-content').find('p', class_='text-muted').find('span',class_='certificate')
            if not Certificate:
                certificate = ' '
            else:
                certificate = Certificate.get_text()
            runtime = movies[index].find(class_='lister-item-content').find('span', class_='runtime').get_text()
            genre = movies[index].find(class_='lister-item-content').find('span', class_='genre').get_text().rstrip()[1:]
            rating = float(movies[index].find(class_='lister-item-content').find('strong').get_text())
            Metascore = movies[index].find(class_='lister-item-content').find('span', class_='metascore favorable')
            if not Metascore:
                metascore = ' '
            else:
                metascore = int(Metascore.get_text().rstrip())
            intro = movies[index].find(class_='lister-item-content').find_all('p', class_='text-muted')[1].get_text()[1:]
            votes = int(movies[index].find(class_='lister-item-content').find('p', class_= 'sort-num_votes-visible').find_all('span')[1]['data-value'])
            Gross = len(movies[index].find(class_='lister-item-content').find('p', class_= 'sort-num_votes-visible').find_all('span'))
            if Gross == 5:
                gross = int(''.join(movies[index].find(class_='lister-item-content').find('p', class_= 'sort-num_votes-visible').find_all('span')[-1]['data-value'].split(',')))
            else:
                gross = ' '
            URL = 'https://imdb.com'+movies[index].find(class_='lister-item-content').find('h3', class_='lister-item-header').find('a', href = True, text=True)['href']
            
            sheet.append([rank, title, year, certificate, runtime, genre, rating, metascore, votes, gross, URL, intro])
        excel.save('IMDb_database.xlsx')
        last = requests.get(url)
        last_soup = BeautifulSoup(last.content,'html.parser')
        page = last_soup.find('div', class_='redesign').find_all(class_='desc')[0].find('a', class_='lister-page-next next-page') or 0
        if not page:
            url = 0
            return True
        else:
            url = 'https://imdb.com'+last_soup.find('div', class_='redesign').find_all(class_='desc')[0].find('a', class_='lister-page-next next-page')['href']
        

In [11]:
# If the page is the last page
last2_url ='https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start=1500&ref_=adv_prv'
last2 = requests.get(last2_url)
last2_soup = BeautifulSoup(last2.content, 'html.parser')
page = last2_soup.find('div', class_='redesign').find_all(class_='desc')[0].find('a', class_='lister-page-next next-page') or 0
if not page:
    next_page = 0
else:
    next_page = 'https://imdb.com'+last2_soup.find('div', class_='redesign').find_all(class_='desc')[0].find('a', class_='lister-page-next next-page')['href']
next_page

'https://imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start=1550'

In [12]:
len(genre_urls)
genre_urls[0]

('Action',
 'https://imdb.com/search/title?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,')

In [13]:
# Test the last page return 0
last_url = 'https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start=1550&ref_=adv_nxt'
last = requests.get(last_url)
last_soup = BeautifulSoup(last.content,'html.parser')
page = last_soup.find('div', class_='redesign').find_all(class_='desc')[0].find('a', class_='lister-page-next next-page') or 0
if not page:
    next_page = 0
else:
    next_page = 'https://imdb.com'+last2_soup.find('div', class_='redesign').find_all(class_='desc')[0].find('a', class_='lister-page-next next-page')['href']
next_page

0

In [ ]:
# Start scrap!
for i in range(21):
    url = genre_urls[i][1]
    print(genre_urls[i][0])
    scrap(url)
    time.sleep(5)